In [1]:
# %load ../snippets/start.py
# system packages
import os, sys
import warnings
warnings.filterwarnings('ignore')

# basic wrangling
import numpy as np
import pandas as pd

# eda tools
import missingno as msno
import pandas_profiling

In [2]:
# %load ../snippets/visual.py
# must go first
%matplotlib inline
%config InlineBackend.figure_format='retina'

# plotting
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_context("poster", font_scale=1.3)

# Update matplotlib defaults to something nicer
mpl_update = {'font.size':16,
              'xtick.labelsize':14,
              'ytick.labelsize':14,
              'figure.figsize':[12.0,8.0],
              'axes.color_cycle':['#0055A7', '#2C3E4F', '#26C5ED', '#00cc66', '#D34100', '#FF9700','#091D32'],
              'axes.labelsize':20,
              'axes.labelcolor':'#677385',
              'axes.titlesize':20,
              'lines.color':'#0055A7',
              'lines.linewidth':3,
              'text.color':'#677385'}
mpl.rcParams.update(mpl_update)

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))


samplesubmission.csv
test.csv
train.csv



In [3]:
import pickle

In [4]:
train_df = pickle.load(open('../proxy/dataset1','rb'))

In [5]:
test_df = pickle.load(open('../proxy/dataset1_test','rb'))

In [6]:
train_df.head(3)

,deadline,state_changed_at,created_at,launched_at,disable_communication,country,currency,goal,backers_count,final_status
0,2009-05-03 06:59:59,2009-05-03 07:00:17,2009-04-24 19:15:07,2009-04-24 19:52:03,0,10,8,20.0,3,1
1,2009-05-15 23:10:00,2009-05-16 00:00:18,2009-04-28 23:10:24,2009-04-29 03:26:32,0,10,8,300.0,2,0
2,2009-05-22 21:26:00,2009-05-22 21:30:18,2009-05-12 21:26:53,2009-05-12 21:39:58,0,10,8,30.0,0,0


In [7]:
# newtrain_df = train_df.assign(Months=(train_df['deadline'].dt.year - train_df['launched_at'].dt.year) * 12 + 
#                 (train_df['deadline'].dt.month - train_df['launched_at'].dt.month))

In [8]:
# newtest_df = test_df.assign(Months=(test_df['deadline'].dt.year - test_df['launched_at'].dt.year) * 12 + 
#                 (test_df['deadline'].dt.month - test_df['launched_at'].dt.month))

In [9]:
newtrain_df = train_df.assign(days=(train_df['deadline'].dt.date - train_df['launched_at'].dt.date)/
                                 np.timedelta64(1, 'D'))

In [10]:
newtest_df = test_df.assign(days=(test_df['deadline'].dt.date - test_df['launched_at'].dt.date)/np.timedelta64(1, 'D'))

In [11]:
newtest_df.columns

Index(['deadline', 'state_changed_at', 'created_at', 'launched_at',
       'disable_communication', 'country', 'currency', 'goal', 'days'],
      dtype='object')

In [12]:
newtrain_df.columns

Index(['deadline', 'state_changed_at', 'created_at', 'launched_at',
       'disable_communication', 'country', 'currency', 'goal', 'backers_count',
       'final_status', 'days'],
      dtype='object')

In [13]:
newtrain_df = newtrain_df[['disable_communication', 'country', 'currency', 'goal','final_status','days']]

In [14]:
newtest_df = newtest_df[['disable_communication', 'country', 'currency', 'goal','days']]

In [35]:
pickle.dump(newtrain_df,open('../proxy/dataset2','wb'))

In [33]:
pickle.dump(newtest_df,open('../proxy/dataset2_test','wb'))

In [17]:
newtest_df.head(3)

,disable_communication,country,currency,goal,days
0,0,20,12,7000.0,40.0
1,0,20,12,35000.0,35.0
2,0,20,12,49500.0,36.0


In [18]:
newtrain_df.head(3)

,disable_communication,country,currency,goal,final_status,days
0,0,10,8,20.0,1,9.0
1,0,10,8,300.0,0,16.0
2,0,10,8,30.0,0,10.0


In [24]:
means = newtrain_df.groupby(['country']).final_status.mean()

In [25]:
means

country
0     0.182447
1     0.224036
2     0.000000
3     0.224490
4     0.286628
5     0.207207
6     0.124823
7     0.157895
8     0.225352
9     0.187500
10    0.332229
Name: final_status, dtype: float64

In [26]:
newtrain_df['mean_target'] = newtrain_df['country'].map(means)

In [28]:
means1 = newtrain_df.groupby(['currency']).final_status.mean()

In [34]:
newtrain_df['mean_target1'] = newtrain_df['currency'].map(means)